In [4]:
import requests
from starlette.applications import Starlette
from starlette.responses import HTMLResponse, JSONResponse
from starlette.middleware.cors import CORSMiddleware
import uvicorn, aiohttp, asyncio

# application init
app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])

subscription_key = "1de9548a300042d1a8c5bf4d1f8233b6"
vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/"
text_recognition_url = vision_base_url + "read/core/asyncBatchAnalyze"
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

@app.route('/')
def index(request):
    return HTMLResponse("Hello World")

@app.route('/getText')
def get_text(request):
    image_url = "https://scontent-lhr3-1.xx.fbcdn.net/v/t1.15752-9/65304028_496860071052571_3668876263856013312_n.jpg?_nc_cat=107&_nc_oc=AQmebaMYcefPNM_P3tVkM6Al1uZ1pGVWKXUhJ9r8UKXphn3zBOAXQMfVWHlB3nUJ5zE&_nc_ht=scontent-lhr3-1.xx&oh=44645c041897dab105c0570f0dcde710&oe=5D81B173"
    data = {'url': image_url}
    response = requests.post(
        text_recognition_url,
        headers=headers,
        json=data)
    
    response.raise_for_status()
    analysis = {}
    poll = True
    
    while (poll):
        response_final = requests.get(response.headers["Operation-Location"], headers=headers)
        analysis = response_final.json()

        if ("recognitionResults" in analysis):
            poll = False
        if ("status" in analysis and analysis['status'] == 'Failed'):
            poll = False
    
    if ("recognitionResults" in analysis):
        texts = [line["text"] for line in analysis["recognitionResults"][0]["lines"]]
    
    return HTMLResponse(str(texts))




if __name__ == '__main__':
    uvicorn.run(app=app, host='0.0.0.0', port=1997)

INFO: Started server process [12150]
INFO: Waiting for application startup.
INFO: Uvicorn running on http://0.0.0.0:1997 (Press CTRL+C to quit)
INFO: ('127.0.0.1', 49618) - "GET /getText HTTP/1.1" 200
INFO: Shutting down
INFO: Waiting for application shutdown.
INFO: Finished server process [12150]
